In [7]:
# import packages

import pandas as pd
import numpy as np
import altair as alt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [6]:
pip install -U scikit-learn

     |████████████████████████████████| 9.1 MB 7.3 MB/s 
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:

# read in data
data = pd.read_csv('adult.csv')

data.head(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [9]:
# important to know shape of data 
data.shape

(32561, 15)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [11]:
# check for missing values
data.isnull().mean().sort_values(ascending=False)

age               0.0
workclass         0.0
fnlwgt            0.0
education         0.0
education.num     0.0
marital.status    0.0
occupation        0.0
relationship      0.0
race              0.0
sex               0.0
capital.gain      0.0
capital.loss      0.0
hours.per.week    0.0
native.country    0.0
income            0.0
dtype: float64

In [12]:
# change "?" to NA values
data[data == '?'] = np.nan

In [13]:
# check for missing values (again)
data.isnull().mean().sort_values(ascending=False)

occupation        0.056601
workclass         0.056386
native.country    0.017905
age               0.000000
fnlwgt            0.000000
education         0.000000
education.num     0.000000
marital.status    0.000000
relationship      0.000000
race              0.000000
sex               0.000000
capital.gain      0.000000
capital.loss      0.000000
hours.per.week    0.000000
income            0.000000
dtype: float64

In [14]:
# drop all rows with NA values
data = data.dropna()

# Could also impute with mode...
# mode_values = data.mode().iloc[0]
# data = data.fillna(mode_values)

In [15]:
data.shape

(30162, 15)

In [16]:
# define target variable and feature variables
X = data.drop(['income'], axis=1)
y = data['income']

In [17]:
# create train/test sets
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size = 0.3, random_state = 90)

In [18]:
cat = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']

for feature in cat:
    enc = LabelEncoder()
    X_tr[feature] = enc.fit_transform(X_tr[feature])
    X_te[feature] = enc.fit_transform(X_te[feature])

In [19]:
# scale the data
ss = StandardScaler()

X_tr = pd.DataFrame(ss.fit_transform(X_tr), columns=X.columns)
X_te = pd.DataFrame(ss.fit_transform(X_te), columns=X.columns)

In [20]:
# check that all categorical variables have been removed
X_tr.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,-0.257925,0.838679,0.001456,1.224504,-0.046913,-0.395750,1.255193,-0.889389,0.385319,0.695708,-0.147962,4.071373,-0.073877,0.264747
1,-0.182098,-0.209074,1.636195,0.175607,-0.440422,-0.395750,1.752587,-0.889389,0.385319,0.695708,-0.147962,-0.218633,0.343975,0.264747
2,-0.030445,1.886433,-0.409512,-0.348842,1.133617,-0.395750,0.757799,-0.889389,-3.216998,0.695708,0.280748,-0.218633,1.597530,0.264747
3,1.258609,-0.209074,0.831980,1.224504,-0.046913,-1.729317,1.006496,-0.265484,0.385319,0.695708,-0.147962,-0.218633,-0.073877,0.264747
4,0.045382,0.838679,-0.630341,-0.873290,0.740107,-0.395750,-0.734382,-0.889389,0.385319,0.695708,1.927749,-0.218633,1.179678,0.264747


In [21]:
# test Logistic Regression with all features
lr = LogisticRegression()

lr.fit(X_tr, y_tr)
y_pred = lr.predict(X_te)

# check model accuracy
accuracy = metrics.accuracy_score(y_te, y_pred)
rounded_accuracy = round(accuracy, 4)

print("Accuracy (All Features):", rounded_accuracy)

Accuracy (All Features): 0.8142


PCA

In [22]:
# create matrix of all variables
x_mx = X_tr

In [ ]:
# center and scale (normalize the data)
# StandardScaler() does the same thing so we don't need to do this
# We WOULD do this if we hadn't used StandardScaler() yet
# x_mx = (x_mx - x_mx.mean())/x_mx.std()
# ss = StandardScaler()
# X = pd.DataFrame(ss.fit_transform(X), columns=X.columns)

In [23]:
# compute principal components
pca = PCA(n_components = x_mx.shape[1])
pca.fit(x_mx)

PCA(n_components=14)

In [24]:
# store proportion of variance explained as a dataframe
pca_var_explained = pd.DataFrame({'Proportion of variance explained': pca.explained_variance_ratio_})

# add component number as a new column
pca_var_explained['Component'] = np.arange(1, 15)

# print
pca_var_explained.head()

,Proportion of variance explained,Component
0,0.148696,1
1,0.100759,2
2,0.082186,3
3,0.079604,4
4,0.073702,5


In [25]:
# add cumulative variance explained as a new column
pca_var_explained['Cumulative variance explained'] = pca_var_explained.iloc[:, 0].cumsum(axis = 0)

# print
pca_var_explained

,Proportion of variance explained,Component,Cumulative variance explained
0,0.148696,1,0.148696
1,0.100759,2,0.249454
2,0.082186,3,0.331640
3,0.079604,4,0.411245
4,0.073702,5,0.484946
5,0.072478,6,0.557425
6,0.069599,7,0.627024
7,0.067797,8,0.694822
8,0.065243,9,0.760064
9,0.060154,10,0.820219


In [26]:
# encode component axis only as base layer
base = alt.Chart(pca_var_explained).encode(
    x= alt.X('Component', scale = alt.Scale(domain=[1, 14]), axis=alt.Axis(values=[1,2,3,4,5,6,7,8,10,11,12,13,14]))
)

# make a base layer for the proportion of variance explained
prop_var_base = base.encode(
    y=alt.Y('Proportion of variance explained', axis=alt.Axis(titleColor='#57A44C'))
)

# make a base layer for the cumulative variance explained
cum_var_base = base.encode(
    y=alt.Y('Cumulative variance explained', axis=alt.Axis(titleColor='#5276A7'))
)

# add points and lines to each base layer
prop_var = prop_var_base.mark_line(
    stroke='#57A44C') + prop_var_base.mark_point(color='#57A44C')

cum_var = cum_var_base.mark_line() + cum_var_base.mark_point()

# layer the layers
var_explained_plot = alt.layer(
    prop_var, cum_var).resolve_scale(y='independent')

# display
var_explained_plot.properties(height = 200, width = 400)

alt.LayerChart(...)

In [27]:
# store the loadings as a data frame with appropriate names
loading_df = pd.DataFrame(pca.components_).transpose().rename(
    columns = {0: 'PC1', 1: 'PC2', 2: 'PC3', 3: 'PC4'} # add entries for each selected component
    ).loc[:, ['PC1', 'PC2', 'PC3', 'PC4']] # slice just components of interest

# add a column with the variable names
loading_df['Variable'] = x_mx.columns.values

# print
# this shows the loading values of each of our variables
loading_df

,PC1,PC2,PC3,PC4,Variable
0,0.316239,0.038751,-0.557908,0.094998,age
1,0.137098,-0.036061,-0.088783,-0.020849,workclass
2,-0.056135,0.165404,0.353635,0.226269,fnlwgt
3,0.109530,-0.597718,0.088932,0.112482,education
4,0.214873,-0.598210,0.177303,0.167165,education.num
5,-0.325872,-0.001940,0.470447,-0.045302,marital.status
6,0.069158,-0.006265,0.374719,0.134365,occupation
7,-0.531507,-0.223402,-0.175295,0.032599,relationship
8,0.168621,-0.038543,0.095233,-0.620893,race
9,0.463801,0.309862,0.323979,-0.042460,sex


In [28]:
# melt from wide to long
loading_plot_df = loading_df.melt(
    id_vars = 'Variable',
    var_name = 'Principal Component',
    value_name = 'Loading'
)

# add a column of zeros to encode for x = 0 line to plot
loading_plot_df['zero'] = np.repeat(0, len(loading_plot_df))

# create base layer
base = alt.Chart(loading_plot_df)

# create lines + points for loadings
loadings = base.mark_line(point = True).encode(
    y = alt.X('Variable', title = ''),
    x = 'Loading',
    color = 'Principal Component'
)

# create line at zero
rule = base.mark_rule().encode(x = alt.X('zero', title = 'Loading'), size = alt.value(0.05))

# layer
loading_plot = (loadings + rule).properties(width = 120)

# show
loading_plot.facet(column = 'Principal Component')

alt.FacetChart(...)

In [29]:
cat = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']

for feature in cat:
    enc = LabelEncoder()
    X[feature] = enc.fit_transform(X[feature])


ss = StandardScaler()
X = pd.DataFrame(ss.fit_transform(X), columns=X.columns)

In [30]:
# reduce data to 4 dimensions
pca = PCA(n_components=4)
pcs = pca.fit_transform(X)

# create dataframe
df_pca = pd.DataFrame(data=pcs, columns=['PC1', 'PC2', 'PC3', 'PC4'])

df_pca.head()

,PC1,PC2,PC3,PC4
0,0.217887,-0.808031,-1.836194,-0.661182
1,-0.121355,0.742021,-3.029424,-0.550313
2,-0.416403,-1.917875,0.679379,-0.388376
3,0.088742,-1.318049,-1.233573,-0.205053
4,-0.735640,1.728807,-0.693450,-1.528792


In [31]:
df_pca.shape

(30162, 4)

In [32]:
# new train/test split
X_tr, X_te, y_tr, y_te = train_test_split(df_pca, y, test_size=0.3, random_state=805)

# test Logistic Regression with all features
lr = LogisticRegression()

lr.fit(X_tr, y_tr)
y_pred = lr.predict(X_te)

# check model accuracy
accuracy = metrics.accuracy_score(y_te, y_pred)
rounded_accuracy = round(accuracy, 4)

print("Accuracy (Using PC1-4):", rounded_accuracy)

Accuracy (Using PC1-4): 0.7999


In [44]:
# create new dummy data
new_data = {
    'age': [50],
    'workclass': [-1.2],
    'fnlwgt': [3],
    'education': [20],
    'education.num': [50],
    'marital.status': [20],
    'occupation': [20],
    'relationship': [2],
    'race': [3],
    'sex': [1],
    'capital.gain': [2],
    'capital.loss': [1],
    'hours.per.week': [6],
    'native.country': [3.4]
}

new_data = pd.DataFrame(new_data)

print(new_data)

   age  workclass  fnlwgt  education  education.num  marital.status   
0   50       -1.2       3          2              5               2  \

   occupation  relationship  race  sex  capital.gain  capital.loss   
0         0.5             2     3    1             2             1  \

   hours.per.week  native.country  
0               6             3.4  


In [45]:
# what about if I had new data?

# Assume new_data is your new data
# standardize new data using the SAME scaler
new_data_scaled = ss.transform(new_data)

# apply PCA using the PCA object from before
new_data_pca = pca.transform(new_data_scaled)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [46]:
# Predict outcomes on the new data
new_data_predictions = lr.predict(new_data_pca)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [47]:
new_data_predictions

array(['<=50K'], dtype=object)